# Classificador Home e Bart - Rede Neural Convolucional

A base de dados de imagens de treinamento e teste estão na pasta raiz dataset_personagens

## Importando bibliotecas

In [3]:
!pip install -q tensorflow==2.16.1

In [4]:
# Importacao desta lib para desativar erro no TensorFlow
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Configurar diretórios de treinamento e teste

In [11]:
train_dir = 'dataset_personagens/training_set'
test_dir = 'dataset_personagens/test_set'

## Configurar o ImageDataGenerator

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,       # Normalizar os pixels para o intervalo [0, 1]
    rotation_range=30,     # Rotação aleatória de até 30 graus
    width_shift_range=0.2, # Deslocamento horizontal
    height_shift_range=0.2,# Deslocamento vertical
    shear_range=0.2,       # Transformação de cisalhamento
    zoom_range=0.2,        # Zoom aleatório
    horizontal_flip=True,  # Inversão horizontal
    fill_mode='nearest'    # Preenchimento dos pixels ausentes
)

In [13]:
test_datagen = ImageDataGenerator(rescale=1.0/255)  # Apenas normalizar os dados de teste

## Carregar imagens das pastas

In [14]:
batch_size = 16
image_size = (64, 64)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'  # Como temos 2 classes (Homer e Bart), usamos 'binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 196 images belonging to 2 classes.
Found 73 images belonging to 2 classes.


## Construir a rede neural convolucional

In [7]:
classificador = Sequential()
classificador.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(MaxPooling2D(pool_size = (2, 2)))

classificador.add(Conv2D(32, (3, 3), activation = 'relu'))
classificador.add(MaxPooling2D(pool_size = (2, 2)))

classificador.add(Flatten())

classificador.add(Dense(units = 4, activation = 'relu'))
classificador.add(Dense(units = 4, activation = 'relu'))
classificador.add(Dense(units = 1, activation = 'sigmoid'))
classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

/home/lucas/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Criar o ImageDataGenerator

In [8]:
gerador_treinamento = ImageDataGenerator(rescale = 1./255, rotation_range=7, 
                                         horizontal_flip = True, shear_range=0.2,
                                         height_shift_range=0.07, zoom_range=0.2)
gerador_teste = ImageDataGenerator(rescale = 1./255)

## Importar base de imagens

In [9]:
base_treinamento = gerador_treinamento.flow_from_directory('dataset_personagens/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 10,
                                                 class_mode = 'binary')
base_teste = gerador_teste.flow_from_directory('dataset_personagens/test_set',
                                            target_size = (64, 64),
                                            batch_size = 10,
                                            class_mode = 'binary')

Found 196 images belonging to 2 classes.
Found 73 images belonging to 2 classes.


## Treinar o modelo

In [11]:
classificador.fit(base_treinamento, steps_per_epoch = 196, epochs = 100,
                         validation_data = base_teste, validation_steps = 73)

Epoch 1/100


/home/lucas/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 18/196 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.5803 - loss: 0.6669

2024-12-06 16:37:03.219053: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/home/lucas/anaconda3/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
/home/lucas/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5815 - loss: 0.6776 - val_accuracy: 0.5753 - val_loss: 0.6825
Epoch 2/100
  7/196 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.6942 - loss: 0.6810

...

2024-12-06 16:38:11.166951: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-12-06 16:38:11.261480: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9857 - loss: 0.0530 - val_accuracy: 0.9726 - val_loss: 0.1424
Epoch 100/100
  1/196 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 1.0000 - loss: 0.0541

2024-12-06 16:38:11.766156: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-12-06 16:38:11.847009: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9851 - loss: 0.0490 - val_accuracy: 0.9452 - val_loss: 0.1275


2024-12-06 16:38:12.347435: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-12-06 16:38:12.431120: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


## Avaliar o modelo

In [16]:
steps = len(base_teste)  # base_teste é o gerador, len(base_teste) retorna o total de lotes
loss, accuracy = classificador.evaluate(base_teste, steps=steps)
print(f"Acurácia no conjunto de teste: {accuracy:.2f}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9709 - loss: 0.0796
Acurácia no conjunto de teste: 0.95
